In [ ]:
-- USE BdcTrigger

SELECT TOP 200 IdDDLEvents, EventDate, EventType, ObjectName, EventDDL,  ApplyDatabaseAudit from BdDDL.dbo.DDLEvents 
		WHERE	DatabaseName = 'bdctRIGGER' 
				AND EventType IN ('CREATE_TABLE', 'ALTER_TABLE', 'RENAME', 'DROP_TABLE' ) 
				AND ApplyDatabaseAudit = 0 
                AND LoginName = 'uJoirRamalho' -- 'TCE\67154832487'
                -- AND IdDDLEvents = 60938
ORDER BY EventDate desc

In [25]:
-- USE BdcTrigger_Audit

SELECT * FROM _Migrate_Table_Audit

(0 rows affected)

IdMigrateTableAudit,IdDDLEvents,EventType,ObjectName,EventDateMigrate,ResultMessage,Success


Total execution time: 00:00:00.019

In [28]:
USE [BdcTrigger]
GO

EXEC master.dbo.sp_MigrateTableAudit_BETA @db = 'bdcTrigger'

-- EXEC master.dbo.sp_CreateTriggerAudit_BETA @db = 'bdcTrigger', @opcao = 0



-- EXEC master.dbo.sp_MigrateTableAudit @db = 'bdcTrigger' --, @opcao = 1

-- EXEC master.dbo.sp_CreateTriggerAudit @db = 'bdcTrigger', @opcao = 1

Commands completed successfully.

In [ ]:
-- sp_Help & sp_helpindex & sp_HelpText
-- EXEC  [BdcTrigger].dbo.sp_Help 'Seg_Operador'

EXEC  [BdcTrigger].dbo.sp_HelpText 'tg_Seg_Operador_audit'

-- EXEC  [BdcTrigger_Audit].dbo.sp_Help 'Seg_Operador'

-- EXEC  [BdcTrigger_Audit].dbo.sp_helpindex 'Seg_Operador'

-- EXEC  [ServerName].[DatabaseName].dbo.sp_HelpText 'storedProcName'

In [ ]:
-- sp_MigrateTableAudit
USE [master]
GO
/****** Object:  StoredProcedure [dbo].[BdAudit]    Script Date: 21/05/2020 11:08:29 ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO

-- 22Junho V1.05

-- EXEC Master.dbo.sp_MigrateTableAudit @db = 'bdcTrigger'

ALTER PROCEDURE [dbo].[sp_MigrateTableAudit]
	@db	VARCHAR(128)
AS
BEGIN
	PRINT 'VERSAO ORIGINAL 9' -- LIMPEZA
	
	SET NOCOUNT ON

	DECLARE	@IdDDLEvents	INT
	DECLARE @EventType		VARCHAR(32)
	DECLARE @Tb				VARCHAR(128)
	DECLARE @TSQL			nVARCHAR(MAX)

	DECLARE @ExecDb 		nVARCHAR(max) = QUOTENAME( @db ) 			+ N'.sys.sp_executesql'
	DECLARE @ExecDbAudit 	nVARCHAR(max) = QUOTENAME( @db + '_audit' ) + N'.sys.sp_executesql'

	DECLARE @TbRename		VARCHAR(128)

	DECLARE @CamposAudit	VARCHAR(MAX) = ''

	DECLARE @Trigger		nVARCHAR(MAX)

	CREATE TABLE #tmpTabelas	( IdDDLEvents INT, EventType VARCHAR(32), ObjectName VARCHAR(128), EventDDL nVARCHAR(max) )
    CREATE TABLE #tmpTrigger    ( TriggerName   VARCHAR(128) )
    CREATE TABLE #tmpCampos	    ( Campos        VARCHAR(128) )


	INSERT INTO #tmpTabelas
		SELECT	IdDDLEvents, EventType, ObjectName, EventDDL
		FROM	BdDDL.dbo.DDLEvents 
		WHERE	DatabaseName = @db
				AND EventType IN ('CREATE_TABLE', 'ALTER_TABLE', 'RENAME', 'DROP_TABLE' ) 
				AND ApplyDatabaseAudit = 0 
		ORDER BY IdDDLEvents


	WHILE ( SELECT COUNT(*) FROM #tmpTabelas ) > 0
        BEGIN
            SELECT	TOP 1 @IdDDLEvents = IdDDLEvents, @EventType = EventType, @Tb = ObjectName, @TSQL = EventDDL
            FROM	#tmpTabelas
            ORDER   BY IdDDLEvents


            IF LEFT ( @tb, 4 ) = 'Tmp_' 
                SET @Tb = RTRIM( SUBSTRING( @Tb, 5, 128 ) )


            SET @TSQL = REPLACE ( @TSQL , 'IDENTITY (1, 1)' , '' )
            SET @TSQL = REPLACE ( @TSQL , 'NOT' , '' )

    
            BEGIN TRY
                IF @EventType IN ( 'ALTER_TABLE', 'DROP_TABLE' )
                    BEGIN
                        IF CHARINDEX('ADD CONSTRAINT', @TSQL ) = 0 AND CHARINDEX('DROP CONSTRAINT', @TSQL ) = 0
                            EXEC	@ExecDbAudit @TSQL
                    END

            
                ELSE IF @EventType IN ( 'RENAME' )
                    BEGIN
                        IF  CHARINDEX('COLUMN', @TSQL ) > 0 -- RENAME COLUMN -- DROP TRIGGER TABELA BASE
                            BEGIN
                                SET @TbRename	= SUBSTRING( @TSQL, CHARINDEX('.', @TSQL ) + 1, LEN( @TSQL ) - CHARINDEX('.', @TSQL ) - CHARINDEX('.', REVERSE( @TSQL ) ) )

                                SET	@Trigger	= 'SELECT o.name FROM sys.triggers AS t INNER JOIN sys.objects AS o  ON t.parent_id = o.object_id WHERE o.name = @TbRename';    

                                DELETE #tmpTrigger

                                INSERT INTO #tmpTrigger
                                    EXEC	@ExecDb @Trigger, N'@TbRename varchar(128)', @TbRename;

                                IF	EXISTS ( SELECT * FROM #tmpTrigger )
                                    BEGIN
                                        SET		@Trigger = 'DROP TRIGGER dbo.tg_' + @TbRename + '_audit';
                            
                                        EXEC	@ExecDb @Trigger
                                    END
                            END

                        EXEC	@ExecDbAudit @TSQL
                    END



                ELSE IF @EventType IN ( 'CREATE_TABLE' ) AND CHARINDEX('Tmp_', @TSQL ) > 0
                    BEGIN                   
                        EXEC	@ExecDbAudit @TSQL	-- Nova tabela Tmp  --	SET		@TSQL = 'SELECT * INTO ' + @db + '_Audit' + '.dbo.Tmp_' + @Tb + ' FROM ' + @db + '.dbo.' + @Tb + ' WHERE 1 = 2';		      

                        
                        SET		@TSQL = 'ALTER TABLE ' + @db + '_Audit' + '.dbo.Tmp_' + @Tb + ' ADD _Audit_Operacao char(1) NOT NULL, _Audit_UsuarioConexao varchar(20) NOT NULL, _Audit_DataConexao datetime NOT NULL';		      

                        EXEC	@ExecDbAudit @TSQL  -- ADD Campos _Audit no final

                        SET		@TSQL = 'SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = @Tb'; 

                        DELETE  #tmpCampos

                        INSERT INTO #tmpCampos
                            EXEC	@ExecDbAudit @TSQL, N'@tb VARCHAR(128)', @Tb; -- GET Campos de _Audit

                        SELECT	@CamposAudit	= @CamposAudit + Campos + ', ' FROM #tmpCampos
                        SET		@CamposAudit	= SUBSTRING( @CamposAudit, 0, LEN( @CamposAudit ));


                        SET		@TSQL = 'INSERT INTO ' + @db + '_Audit' + '.dbo.Tmp_' + @Tb + ' ( ' + @CamposAudit + ' ) SELECT ' + @CamposAudit + ' FROM ' + @db + '_Audit' + '.dbo.' + @Tb + ' WITH (HOLDLOCK TABLOCKX)';

                        EXEC	@ExecDbAudit @TSQL  -- INSERT tabela atual de _Audit na Tmp_*
               
                        SET	@Trigger	= 'SELECT o.name FROM sys.triggers AS t INNER JOIN sys.objects AS o  ON t.parent_id = o.object_id WHERE o.name = @Tb';    

                        DELETE  #tmpTrigger

                        INSERT INTO #tmpTrigger
                            EXEC	@ExecDb @Trigger, N'@Tb varchar(128)', @Tb;

                        IF	EXISTS ( SELECT * FROM #tmpTrigger )
                            BEGIN
                                SET		@Trigger = 'DROP TRIGGER dbo.tg_' + @Tb + '_audit';
                
                                EXEC	@ExecDb @Trigger
                            END
                END

                --- FINAL
                SET		@TSQL = N'UPDATE BdDDL.dbo.DDLEvents SET ApplyDatabaseAudit = 1 WHERE IdDDLEvents = @Id;';

                EXEC	@ExecDb @TSQL, N'@Id int', @IdDDLEvents;

                DELETE FROM  #tmpTabelas WHERE  IdDDLEvents = @IdDDLEvents
            END TRY
            BEGIN CATCH
                PRINT	ERROR_MESSAGE()
            END CATCH
        END
END


In [13]:
-- RESTORE   BDC

USE [master]
ALTER DATABASE [BdcTrigger] SET SINGLE_USER WITH ROLLBACK IMMEDIATE
    RESTORE DATABASE [BdcTrigger] FROM  DISK = N'\\fsbkp\sql\PRD-SQL02S\Bdc\Bdc.BAK' WITH  FILE = 1,  MOVE N'Bdc' TO N'E:\Feature\BdcTrigger.mdf',  MOVE N'Bdc_log' TO N'E:\Feature\BdcTrigger_log.ldf',  replace, NOUNLOAD,  STATS = 50
ALTER DATABASE [BdcTrigger] SET MULTI_USER

ALTER DATABASE [BdcTrigger_Audit] SET SINGLE_USER WITH ROLLBACK IMMEDIATE
RESTORE DATABASE [BdcTrigger_Audit] FROM  DISK = N'\\fsbkp\sql\PRD-SQL02S\Bdc_Audit\Bdc_Audit.BAK' WITH  FILE = 1,  MOVE N'Bdc_Audit' TO N'E:\Feature\BdcTrigger_Audit.mdf',  MOVE N'Bdc_Audit_log' TO N'E:\Feature\BdcTrigger_Audit_log.ldf',  replace, NOUNLOAD,  STATS = 50
ALTER DATABASE [BdcTrigger_Audit] SET MULTI_USER

-- EXEC master.dbo.sp_CreateTriggerAudit @db = 'BdcTrigger', @opcao = 1

UPDATE BdDDL.dbo.DDLEvents SET ApplyDatabaseAudit = 1 WHERE ApplyDatabaseAudit = 0 -- IdDDLEvents = 62876

Nonqualified transactions are being rolled back. Estimated rollback completion: 0%.

Nonqualified transactions are being rolled back. Estimated rollback completion: 100%.

51 percent processed.

100 percent processed.

Processed 6400 pages for database 'BdcTrigger', file 'Bdc' on file 1.

...

Processed 1696 pages for database 'BdcTrigger_Audit', file 'Bdc_Audit' on file 1.

Processed 2 pages for database 'BdcTrigger_Audit', file 'Bdc_Audit_log' on file 1.

RESTORE DATABASE successfully processed 1698 pages in 0.125 seconds (106.066 MB/sec).

(224 rows affected)

Total execution time: 00:00:03.979

In [ ]:
-- sp_CreateTriggerAudit
USE master
GO

-- 18Junho20 V1.05

-- EXEC master.dbo.sp_CreateTriggerAudit @db = 'bdcTrigger', @opcao = 1

ALTER PROCEDURE [dbo].[sp_CreateTriggerAudit] 
	@db		VARCHAR(128),
	@opcao	tinyint = 0 -- 0-Apenas tabelas alteradas; 1 - Geral
AS
BEGIN
	PRINT 'VERSAO ORIGINAL 1' -- _Audit_

	SET NOCOUNT ON

	DECLARE @Tb				VARCHAR(128)
	DECLARE @NomeTrigger	VARCHAR(128)
	DECLARE @NomeTbAudit	VARCHAR(128)

	DECLARE @CamposAudit	VARCHAR(MAX)
	DECLARE @CamposSelect	VARCHAR(MAX)

	DECLARE @exec			nVARCHAR(MAX)
	DECLARE @TSQL			nVARCHAR(MAX)

	DECLARE @Trigger		nVARCHAR(MAX)


	
	SET	@exec = QUOTENAME( @db ) + N'.sys.sp_executesql' -- Banco BASE


	-- TABELAS
	IF	OBJECT_ID('tempdb..#tmpTabelas') IS NOT NULL
		DROP TABLE #tmpTabelas

	CREATE TABLE #tmpTabelas	( Tb VARCHAR(128), NomeTrigger VARCHAR(128), NomeTbAudit VARCHAR(128) )

	IF @opcao = 0 -- Gerar triggers APENAS tabelas que foram atualizadas
		SET		@TSQL = 'SELECT	t.name, tg.name, ( SELECT ta.name from ' + quotename( @db + '_Audit' ) + N'.sys.tables  ta WHERE ta.name = t.name) FROM ' + quotename(@db) + N'.sys.tables t LEFT JOIN  sys.triggers tg ON tg.parent_id = t.object_id	WHERE ( t.modify_date > tg.modify_date ) OR tg.name IS NULL'
	ELSE -- Gerar triggers para todas as tabelas
		SET		@TSQL = 'SELECT	t.name, tg.name, ( SELECT ta.name from ' + quotename( @db + '_Audit' ) + N'.sys.tables  ta WHERE ta.name = t.name) FROM ' + quotename(@db) + N'.sys.tables t LEFT JOIN  sys.triggers tg ON tg.parent_id = t.object_id'


	INSERT INTO #tmpTabelas
		EXEC	@exec @TSQL, N'@db VARCHAR(128)', @db;


	-- LIMPAR de #tmp Trigger de auditoria e do Flyway
	DELETE FROM  #tmpTabelas WHERE ( NomeTrigger LIKE '%_IR' ) OR ( Tb = '_flyway_schema_history_Audit' )

--	SELECT * FROM #tmpTabelas

	WHILE ( SELECT COUNT(*) FROM #tmpTabelas ) > 0
		BEGIN
			SET @CamposAudit	= ''
			SET @CamposSelect	= ''


			SELECT TOP 1 @Tb = Tb, @NomeTrigger = NomeTrigger, @NomeTbAudit = NomeTbAudit FROM #tmpTabelas	 ORDER BY Tb

			DELETE FROM  #tmpTabelas WHERE  Tb = @Tb


			-- CAMPOS
			IF OBJECT_ID('tempdb..#tmpCampos') IS NOT NULL
				DROP TABLE #tmpCampos

			CREATE TABLE #tmpCampos	( Campos VARCHAR(128) )

			SET		@TSQL = 'SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = @Tb'; 

			INSERT INTO #tmpCampos
				EXEC	@exec @TSQL, N'@tb VARCHAR(128)', @Tb;

			SELECT	@CamposAudit	= @CamposAudit + Campos + ', ' FROM #tmpCampos
			SET		@CamposAudit	= SUBSTRING( @CamposAudit, 0, LEN( @CamposAudit ));

			SET		@CamposAudit	= @CamposAudit + ', _Audit_Operacao, _Audit_UsuarioConexao, _Audit_DataConexao'

			-- CREATE Tb_Audit From TbBase
			IF @NomeTbAudit IS NULL 
				BEGIN
					SET		@TSQL = 'SELECT * INTO ' + @db + '_Audit' + '.dbo.' + @Tb + ' FROM ' + @db + '.dbo.' + @Tb + ' WHERE 1 = 2';		      
	
					EXEC	@exec @TSQL
		

					-- ADD campos de audit
					SET		@TSQL = 'ALTER TABLE ' + @db + '_Audit' + '.dbo.' + @Tb + ' ADD _Audit_Operacao char(1) NOT NULL, _Audit_UsuarioConexao varchar(20) NOT NULL, _Audit_DataConexao datetime NOT NULL';		      

					EXEC	@exec @TSQL
				END


			IF @NomeTrigger IS NULL
				SET @Trigger = REPLACE ( 'CREATE TRIGGER [dbo].[tg_TableName_Audit] On [dbo].[TableName]', 'TableName' , @Tb ) + CHAR(10)
			ELSE
				SET @Trigger = REPLACE ( 'ALTER TRIGGER [dbo].[tg_TableName_Audit] On [dbo].[TableName]', 'TableName' , @Tb ) + CHAR(10)


			SET @Trigger = @Trigger + 'FOR UPDATE, DELETE' + CHAR(10)

			SET @Trigger = @Trigger + 'AS' + CHAR(10)

			SET @Trigger = @Trigger + REPLACE ( 'INSERT INTO [BdBase_Audit].[dbo].[TableName]', 'TableName' , @Tb ) 

			SET @Trigger = REPLACE ( @Trigger, 'BdBase' , @db )

			SET @Trigger = @Trigger + REPLACE ( ' ( Campos', 'Campos' , @CamposAudit ) + ' )' + CHAR(10)

			SET @CamposSelect = @CamposAudit

			SET @CamposSelect = REPLACE ( '	SELECT Campos', 'Campos' , @CamposSelect )

			SET @CamposSelect = REPLACE ( @CamposSelect, 'IdSessaoOperacao,',	    '( SELECT TOP 1 IdSessaoOperacao FROM inserted ),' )
			SET @CamposSelect = REPLACE ( @CamposSelect, '_Audit_Operacao,',		'IIF(EXISTS(SELECT * FROM Inserted),''U'',''D''),' )
			SET @CamposSelect = REPLACE ( @CamposSelect, '_Audit_UsuarioConexao,',	'SUSER_SNAME(),' )
			SET @CamposSelect = REPLACE ( @CamposSelect, '_Audit_DataConexao',		'GETDATE()' ) + ' FROM deleted' + CHAR(10)


			SET @Trigger = @Trigger + @CamposSelect


			BEGIN TRY
				EXEC	@exec @Trigger		
			END TRY
			BEGIN CATCH
			 	PRINT	@trigger

				PRINT	ERROR_MESSAGE()
			END CATCH

		END
END

In [ ]:
USE [BdcTrigger]
GO

EXEC master.dbo.sp_MigrateTableAudit_BETA @db = 'bdcTrigger'

EXEC master.dbo.sp_CreateTriggerAudit_BETA @db = 'bdcTrigger', @opcao = 0

--ALTER TABLE dbo.Seg_OperadorInternoSetor SET (LOCK_ESCALATION = TABLE)


-- SELECT *  FROM [BdcTrigger_Audit].[dbo].[Seg_Operador]

-- SELECT TOP 1000 *  FROM [BdcTrigger_Audit].[dbo].[_Migrate_Table_Audit] ORDER BY IdMigrateTableAudit DESC


/*
SELECT TOP 200 [IdDDLEvents]
      ,[EventDate]      ,[EventType]      ,[EventDDL]      ,[DatabaseName]      ,[ObjectName]
--      ,[HostName]
      ,[ProgramName]      ,[LoginName]      ,[ApplyDatabaseAudit]
  FROM [BdDDL].[dbo].[DDLEvents]
  WHERE DatabaseName = 'BdcTrigger' AND EventType IN ('CREATE_TABLE', 'ALTER_TABLE', 'RENAME', 'DROP_TABLE' ) 
  order by IdDDLEvents DESC
*/


In [1]:
USE [PROCESSOTrigger_Audit]
GO
-- USE [BdcTrigger_Audit]
-- GO

/****** Object:  Table [dbo].[_Migrate_Table_Audit]    Script Date: 22/06/2020 16:50:43 ******/
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

SET ANSI_PADDING ON
GO

CREATE TABLE [dbo].[_Migrate_Table_Audit](
	[IdMigrateTableAudit] [int] IDENTITY(1,1) NOT NULL,
	[IdDDLEvents] [int] NOT NULL,
	[EventType] [varchar](32) NOT NULL,
	[ObjectName] [varchar](128) NOT NULL,
	[EventDateMigrate] [datetime] NOT NULL,
	[ResultMessage] [varchar](2048) NOT NULL,
	[Success] [bit] NOT NULL,
 CONSTRAINT [PK__MigrateTableAudit] PRIMARY KEY CLUSTERED 
(
	[IdMigrateTableAudit] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON) ON [PRIMARY]
) ON [PRIMARY]

GO

SET ANSI_PADDING ON
GO

ALTER TABLE [dbo].[_Migrate_Table_Audit] ADD  CONSTRAINT [DF__Migrate_Table_Audit_IdDDLEvents]  DEFAULT ((0)) FOR [IdDDLEvents]
GO

ALTER TABLE [dbo].[_Migrate_Table_Audit] ADD  CONSTRAINT [DF__Migrate_Table_Audit_EventDateMigrate]  DEFAULT (getdate()) FOR [EventDateMigrate]
GO

ALTER TABLE [dbo].[_Migrate_Table_Audit] ADD  CONSTRAINT [DF__Migrate_Table_Audit_Success]  DEFAULT ((1)) FOR [Success]
GO


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.605

In [15]:
EXEC master.dbo.sp_CreateTriggerAudit_BETA @db = 'bdcTrigger', @opcao = 0

*** BETA V1.07 - 01Julho20

EventType,ObjectName,ResultMessage,Success


Total execution time: 00:00:00.746

In [ ]:
---------- Reset SQL user account guids ---------------------

USE BdcTrigger

DECLARE @UserName nvarchar(255) 

DECLARE orphanuser_cur cursor for 
      SELECT UserName = su.name 
      FROM sysusers su JOIN sys.server_principals sp ON sp.name = su.name
      WHERE issqluser = 1 AND (su.sid IS NOT NULL AND su.sid <> 0x0) AND suser_sname(su.sid) is null 
      ORDER BY su.name 
OPEN orphanuser_cur 
FETCH NEXT FROM orphanuser_cur INTO @UserName 

WHILE (@@fetch_status = 0)
BEGIN 
        PRINT @UserName + ' user name being resynced' 
        EXEC sp_change_users_login 'Update_one', @UserName, @UserName 
FETCH NEXT FROM orphanuser_cur INTO @UserName 
END 

CLOSE orphanuser_cur 
DEALLOCATE orphanuser_cur
---

---------- Reset SQL user account guids ---------------------
USE BdcTrigger_Audit

--DECLARE @UserName nvarchar(255) 
DECLARE orphanuser_cur cursor for 
      SELECT UserName = su.name 
      FROM sysusers su JOIN sys.server_principals sp ON sp.name = su.name
      WHERE issqluser = 1 AND (su.sid IS NOT NULL AND su.sid <> 0x0) AND suser_sname(su.sid) is null 
      ORDER BY su.name 
OPEN orphanuser_cur 
FETCH NEXT FROM orphanuser_cur INTO @UserName 

WHILE (@@fetch_status = 0)
BEGIN 
        PRINT @UserName + ' user name being resynced' 
        EXEC sp_change_users_login 'Update_one', @UserName, @UserName 
FETCH NEXT FROM orphanuser_cur INTO @UserName 
END 

CLOSE orphanuser_cur 
DEALLOCATE orphanuser_cur


In [ ]:
-- RENAME _Audits
-- SELECT  'EXECUTE sp_rename ' + quotename(name) + ', ' + quotename( SUBSTRING( name, 1, LEN( name ) - 6 ) ) FROM sys.tables 